# Data Science Nigeria Staff Promotion Compitition
### Predicting staff that are likely to be promoted based on defined personal and performance parameters

In [1]:
import os
for dirname, _, filenames in os.walk('C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/'):
#for dirname, _, filenames in os.walk('C:\\Users\\Arthur\\data-projects\\DataScienceNigeria_ML_Course_Practice'):   
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/ds_nigeria_ml_course_notebook.ipynb
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/.ipynb_checkpoints\ds_nigeria_ml_course_notebook-checkpoint.ipynb
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets\sample_submission2.csv
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets\test.csv
C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets\train.csv


In [2]:
# Graphing imports
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.base import BaseEstimator, TransformerMixin
#from sklearn.datasets import fetch_california_housing
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Ridge, RidgeCV, LogisticRegressionCV, LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.compose import ColumnTransformer

import xgboost as xgb

ModuleNotFoundError: No module named 'sklearn.impute'

# EDA

In [9]:
data = pd.read_csv('C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets/train.csv')
data.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [10]:
print(data.columns)
print(data.No_of_previous_employers.unique())
print(data.Previous_IntraDepartmental_Movement.unique())
print(data.Past_Disciplinary_Action.unique())
print(data.Marital_Status.unique())
print(data.Foreign_schooled.unique())
print(data.State_Of_Origin.unique())
print(data.Year_of_recruitment.unique())
print(data.Channel_of_Recruitment.unique())
print(data.Year_of_birth.unique())
print(data.Gender.unique())
print(data.Qualification.unique())
print(data.Division.unique())

Index(['EmployeeNo', 'Division', 'Qualification', 'Gender',
       'Channel_of_Recruitment', 'Trainings_Attended', 'Year_of_birth',
       'Last_performance_score', 'Year_of_recruitment', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'State_Of_Origin',
       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not'],
      dtype='object')
['0' '1' '4' '3' '2' 'More than 5' '5']
['No' 'Yes']
['No' 'Yes']
['Married' 'Single' 'Not_Sure']
['No' 'Yes']
['ANAMBRA' 'KATSINA' 'NIGER' 'AKWA IBOM' 'ENUGU' 'OYO' 'LAGOS' 'ONDO'
 'KWARA' 'EDO' 'BAUCHI' 'TARABA' 'RIVERS' 'KADUNA' 'PLATEAU' 'EKITI'
 'BORNO' 'IMO' 'CROSS RIVER' 'KANO' 'FCT' 'OGUN' 'GOMBE' 'DELTA' 'BENUE'
 'ZAMFARA' 'OSUN' 'ABIA' 'KEBBI' 'ADAMAWA' 'NASSARAWA' 'SOKOTO' 'KOGI'
 'JIGAWA' 'BAYELSA' 'YOBE' 'EBONYI']
[2011 2015 2012 2009 2014 2013 2016 2003 2010 2008 2018 2017 2007 2002
 2005 2006 2000 2004 2001 199

In [ ]:
print(data.isnull().values.any())
print(data.shape)

In [11]:
X = data
y = data.Promoted_or_Not
z = data.EmployeeNo
print(X.shape, y.shape, z.shape)
X = X.drop(labels=['Promoted_or_Not', 'EmployeeNo'], axis=1) # dropped EmployeeNo since I cant categorize it
print(X.shape, y.shape, z.shape)

(38312, 19) (38312,)
(38312, 17) (38312,)


In [27]:
#import winsound
#winsound.Beep(4300, 1000)

In [ ]:
X.head()    

In [ ]:
X.info()

In [ ]:
print(X.isnull().values.any())
print(y.isnull().values.any())
print(z.isnull().values.any())

In [ ]:
imp = SimpleImputer(strategy='most_frequent')
X_imputed = imp.fit_transform(X)
X_imputed

In [ ]:
X_imputed = pd.DataFrame.from_records(X_imputed)
X_imputed.columns = X.columns
print(X_imputed.columns is X.columns)

In [ ]:
print(X.isnull().values.any())
print(X_imputed.isnull().values.any())
print(y.isnull().values.any())
print(z.isnull().values.any())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=0)
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
print(numerical_cols)
categorical_cols = [cname for cname in X.columns if X[cname].dtype == 'object']
print(categorical_cols)
print(len(numerical_cols) + len(categorical_cols))

In [ ]:
#Transform nan values using SimpleImputer -- a
#Transform numerical cols using StandardScaler -- b
#Transform categorical cols using OneHotEncoder -- c  
#[steps b and c can be done using FeatureUnion or ColumnTransformer]
l_model = LogisticRegressionCV()
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
col_transformer = ColumnTransformer([('scaler', scaler, numerical_cols),
                                     ('ohe', ohe, categorical_cols)],
                                    sparse_threshold=0)
pipe = Pipeline([('col_transformer', col_transformer),
                 ('model', l_model)],
                verbose=True)

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
test_data = pd.read_csv('C:/Users/Arthur/data-projects/DataScienceNigeria_ML_Course_Practice/datasets/train.csv')
test_data.head()

In [ ]:
test_data.info()

In [ ]:
print(test_data.isnull().values.any())
print(test_data.shape)

In [ ]:
test_X = test_data
#y = data.Promoted_or_Not
test_z = test_data.EmployeeNo
print(test_X.shape, test_z.shape)
test_X = test_X.drop(labels='EmployeeNo', axis=1) # dropped EmployeeNo since I cant categorize it
print(test_X.shape, test_z.shape)

In [ ]:
#imp = SimpleImputer(strategy='most_frequent')
#X_imputed = imp.fit_transform(X)
test_X_imputed = imp.transform(test_X)
test_X_imputed = pd.DataFrame.from_records(test_X_imputed, columns=test_X.columns)
#test_X_imputed.columns = test_X.columns
print(test_X_imputed.columns is test_X.columns)

In [ ]:
print(test_X_imputed.isnull().values.any())
print(test_X_imputed.shape, X_imputed.shape)

In [ ]:
test_X_imputed.head()

In [ ]:
test_pred = pipe.predict(test_X_imputed)
test_pred

In [ ]:
output = pd.DataFrame({'EmployeeNo': test_z, 'Promoted_or_Not': test_pred})
output.to_csv('submission_1.csv', index=False)